In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math 

## 关于半个 Attention 矩阵的问题，
训练和推理过程要分开来看，
训练过程，为了锻炼模型的预测下一个 token 的能力，自然只会让模型只“看到”其之前的 token，这是生成任务与翻译任务的区别，
而推理过程中，理论上来讲，用户输入的内容，是可以双向计算 Attention 的，
论文《What Language Model Architecture and Pretraining  Objective Work Best for Zero-Shot Generalization》对三中机制
* Encoder-Decoder
* Decoder only （就是 Causal）
* Decoder only + 特殊的 Attention，就是输入部分，双向计算，生成部分单向

In [44]:
mask = torch.tril(torch.ones(10, 10))
attention = torch.randn(1, 10, 10)
after = attention * mask
#attention_ = attention_.masked_fill(mask == 0, float('-inf'))
sums = after.sum(-1, keepdim=True)#-1是最里面那一层，keepDim=true 表示保持在原来的维度上，每一行加起来还是一行，而不是一列，一列的就缩成一行了，缩的方向就不对了
after2 = after / sums #保证每行加起来是1，也算做了softmax
mask,attention,after,sums,after2


(tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[[-1.5107, -1.2339,  0.3342, -1.0417, -0.1341, -0.4620,  0.3182,
           -0.4359,  0.9680, -2.0363],
          [ 1.5530, -1.3971, -0.9803,  0.3606,  0.2158, -0.5770, -0.9402,
            0.0410,  0.4711,  1.4336],
          [-0.2862,  0.3326,  0.3727,  0.1386,  0.8457, -0.1808, -0.8695,
           -0.8945, -1.4147, -1.5214],
          [ 1.2778,  2.2090, -1.2422,  2.3947, -1.0153, -1.0643, -0.9961,
           -0.1322, -0.2122,  1.2000],
          [-1.2890, -0.4229, -0.5525

In [45]:
#attention = torch.randn(1, 10, 10)
mask = torch.tril(torch.ones(100, 100))  #固定参数的缓存
attention3 = attention.masked_fill(mask[:10, :10] == 0, float('-inf'))
result = F.softmax(attention3, dim=-1)
mask, attention3, result

(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]]),
 tensor([[[-1.5107,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf,    -inf,    -inf],
          [ 1.5530, -1.3971,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf,    -inf,    -inf],
          [-0.2862,  0.3326,  0.3727,    -inf,    -inf,    -inf,    -inf,
              -inf,    -inf,    -inf],
          [ 1.2778,  2.2090, -1.2422,  2.3947,    -inf,    -inf,    -inf,
              -inf,    -inf,    -inf],
          [-1.2890, -0.4229, -0.5525, -0.0515, -0.3217,    -inf,    -inf,
              -inf,    -inf,    -inf],
          [-0.3074, -0.1412,  1.3545, -0.5640, -1.0317,  0.1701,    -inf,
              -inf,    -inf,    -inf],
          [ 0.7267,  0.9915,  0.6307, -0.9482, -0.1192, -2.2

In [46]:
print(float('-inf'))
print(-torch.inf)

-inf
-inf


In [47]:
# dropout attention weights
layer = nn.Dropout(p=0.5)
layer
out = layer(mask)
out,mask # Dropout 在丢弃一部分神经元的同时，会对保留下来的神经元进行缩放，即乘以 1/(1-p)，以保持其期望值不变,如果p=0.5，那么保留下来的神经元会乘以2


(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [2., 0., 0.,  ..., 0., 0., 0.],
         [2., 2., 0.,  ..., 0., 0., 0.],
         ...,
         [2., 0., 0.,  ..., 2., 0., 0.],
         [2., 2., 2.,  ..., 2., 2., 0.],
         [2., 0., 2.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]]))

In [48]:
x = torch.tensor([-2.0, 1])
w = torch.tensor([[2, 1.0], [0, 3]])
y = torch.matmul(x, w)
y #x = torch.tensor([-12.0, 1])

tensor([-4.,  1.])

# 表达式要包括含\$\$之间

$$\left[ 
\begin{array}{cc}
        1 & 0 \\
        0 & 2
    \end{array}
\right]

\left[
\begin{array}{c}
x_1 \\
x_2
\end{array}
\right] $$


$$ \left[ 
\begin{array}{cc}
        1 & 0 \\
        0 & 2
    \end{array}
\right]

\left[
\begin{array}{c}
x_1 \\
x_2
\end{array}
\right] $$

$x = {-b \pm \sqrt{b^2-4ac} \over 2a}$
$F = G{Mm \over r^2}$ 
$\alpha, \beta, \gamma, \Gamma, \pi$


$\sqrt{x}, \sqrt[3]{x}, \sqrt[n]{x}$

$\left[ 
\begin{matrix}
   a & b & c\\
   d & e & f
\end{matrix}

$$f(n) = 
\begin{cases}
n/2, & \text{if } n \text{ is even} \\
3n+1, & \text{if } n \text{ is odd}
\end{cases}
$$


$$\left( \frac{x}{y} \right), \quad 

\left[ f(x) \right]

, \quad \left| x \right|$$


In [49]:
w1 = torch.tensor([[2, 0.0], [1, 3]])
y1 = torch.matmul(x, w1)
y1


tensor([-3.,  3.])

#
向量[-2,1]与矩阵 $ \left[ \begin{array}{cc} 2,1\\0,3 \end{array} \right] $ 相乘，

# 点积，到底是什么？
按照Transformer 的架构的原理，用点积的结果表示两个向量的相似度或者说相关性，那按照这个逻辑，在点积之前就应该计算 softmax，这样点积的结果才更加能够表示两个 token 之间的关系，而实际 Transformer 架构中是先计算 Attention，再进行 softmax，这样做是否有问题，Transformer 的架构在这点上是否有调整的可能？

点积能够比较两个向量的特征，这个特征不是一种概率分布，就是在各个维度上的取值，
而 softmax，是一种量化的手段，模型的某些层级上可以去“考虑”某些维度是重要的，某些维度是不重要的，softmax 不是用来调整模型的特征的取值，只能选择而不是调整改变


大语言模型是"概率模型"，不是因为其内部表示是概率分布，而是因为：
训练目标：最小化基于概率的损失函数（交叉熵）
输出形式：最终输出是词汇表上的概率分布
生成过程：基于概率进行序列生成
内部机制：使用注意力权重的概率分布来加权信息


Normalization（归一化）的目的是为了改变数据的分布，使其更利于模型训练。而 Softmax 的目的是将一个向量“压缩”为概率分布，使其所有元素之和为1。
Normalization 是一个数据预处理/特征工程的技术，它通过对原始数据进行线性或非线性变换，将数据缩放到一个特定的尺度或范围内（如 [0, 1] 或 均值为0，方差为1），但不改变数据本身的分布结构（线性变换不改变）。


加速模型收敛：在梯度下降中，如果特征尺度差异巨大，损失函数的等高线会是椭球状，导致优化路径呈“之字形”，收敛缓慢。归一化后，等高线更接近圆形，优化路径更直接。


Softmax 是一个数学函数，它接收一个任意实数的向量（通常是神经网络的原始输出，称为 logits），并将其“压缩”成一个概率分布。这个概率分布的所有元素都在 (0, 1) 之间，且所有元素之和为 1。


Normalization是针对不同数据的上的同一个特征，
softmax 一般是针对同一个数据的不同特征，